In [6]:
import cv2
import numpy as np
import os

In [7]:
# Callback “vacío” para los trackbars
def nothing(x):
    pass

In [8]:
# --- Configuración de ventana y trackbars para ROI ---
cv2.namedWindow('Detección con NMS')

# Crear sliders: X, Y, W, H
cv2.createTrackbar('X', 'Detección con NMS', 0, 1280, nothing)
cv2.createTrackbar('Y', 'Detección con NMS', 0, 720, nothing)
cv2.createTrackbar('W', 'Detección con NMS', 500, 1280, nothing)
cv2.createTrackbar('H', 'Detección con NMS', 300, 720, nothing)

In [9]:
# --- Inicializar vídeo y sustractor de fondo ---
cap = cv2.VideoCapture('highway.mp4')
object_sub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Leer parámetros de la ROI desde los sliders
    x = cv2.getTrackbarPos('X', 'Detección con NMS')
    y = cv2.getTrackbarPos('Y', 'Detección con NMS')
    w = cv2.getTrackbarPos('W', 'Detección con NMS')
    h = cv2.getTrackbarPos('H', 'Detección con NMS')

    # Asegurar que la ROI quede dentro de los límites del frame
    x = min(max(x, 0), frame.shape[1] - 1)
    y = min(max(y, 0), frame.shape[0] - 1)
    w = min(w, frame.shape[1] - x)
    h = min(h, frame.shape[0] - y)

    # Extraer ROI
    roi = frame[y:y+h, x:x+w]

    # 1) Detectar movimiento en la ROI
    mask = object_sub.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    
    # 2) Encontrar contornos y generar bounding boxes + scores (área)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes, scores = [], []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 500:  # Filtrar ruido pequeño
            continue
        bx, by, bw, bh = cv2.boundingRect(cnt)
        # Convertir coordenadas de ROI a coordenadas globales
        boxes.append([bx + x, by + y, bw, bh])
        scores.append(float(area))

    # 3) Aplicar NMS
    indices = []
    if len(boxes) > 0:
        indices = cv2.dnn.NMSBoxes(boxes, scores,
                                  score_threshold=0.0,
                                  nms_threshold=0.4)
    
    # 4) Dibujar resultados
    # - Dibujar ROI en azul
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    # - Dibujar todas las detecciones retenidas por NMS en verde
    if len(indices) > 0:
        for i in indices.flatten():
            bx, by, bw, bh = boxes[i]
            cv2.rectangle(frame, (bx, by), (bx + bw, by + bh), (0, 255, 0), 2)

    # Mostrar frame resultante
    cv2.imshow('Detección con NMS', frame)

    # Salir con ESC
    if cv2.waitKey(30) == 27:
        break

cap.release()
cv2.destroyAllWindows()